In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, MultiPoint, LineString, Polygon, MultiPolygon, MultiLineString
import fiona
from shapely.wkt import loads
from collections import defaultdict
from functools import reduce
from shapely import ops
from shapely.ops import split, snap, nearest_points, shared_paths
from shapely.ops import linemerge, unary_union, polygonize, polygonize_full, cascaded_union
from shapely.affinity import scale

In [2]:
def explode_geometries(this_gdf):
    this_exploded_geometries = this_gdf.explode().reset_index()#.rename(columns = {0: 'split_geometry'})
    this_exploded_geometries_merged = this_exploded_geometries.merge(this_gdf.drop('geometry', axis=1), left_on='level_0', right_index=True)
    this_exploded_geometries_merged = this_exploded_geometries_merged.drop('level_0', axis=1).drop('level_1', axis=1)
    this_exploded_gdf = gpd.GeoDataFrame(this_exploded_geometries_merged, geometry=this_exploded_geometries_merged.geometry)
    return this_exploded_gdf

In [3]:
def cut_polygon_by_line(polygon, line):
    scaled_line = scale(line, xfact=1.05, yfact=1.05, origin=line.centroid)
    merged = linemerge([polygon.boundary, scaled_line])
    borders = unary_union(merged)
    split_polygons = polygonize(borders)
    return list(filter(None, [str(s) for s in split_polygons]))

In [4]:
def check_for_intersection(polygon, line):
    if polygon.intersects(line) and all([sh.is_empty for sh in shared_paths(polygon.boundary, line)]):
        return True

In [5]:
uganda = gpd.read_file('./uganda.shp')

In [6]:
admin3 = gpd.read_file('./uga_admbnda_adm3_UBOS_v5.shp')

In [7]:
admin3_clipped = gpd.overlay(admin3, uganda, how='intersection')
#trim the admin boundary to the national border, but be wary of little slices left behind 
#that create multipolygons and will screw everything up later

In [8]:
admin3_clipped_exploded = explode_geometries(admin3_clipped)

In [9]:
admin3_outer_boundary_out = admin3_clipped.dissolve(by='ADM0_EN')
#dissolve to make a single poly for the whole country 

In [10]:
admin3_outer_boundary_poly = ops.unary_union([Polygon(p.buffer(0).exterior) for p in admin3_outer_boundary_out['geometry']])
# this eats any holes inside the primary outer boundary polygon

In [11]:
point_counter = defaultdict(int)
admin3_outer_boundary_poly_coords = list(admin3_outer_boundary_poly.exterior.coords)
admin3_outer_boundary_poly_coords_length = len(admin3_outer_boundary_poly_coords)

print('checking ', admin3_outer_boundary_poly_coords_length, ' admin3_outer_boundary_poly_coords' )

for eachpoint in [Point(p) for p in admin3_outer_boundary_poly_coords]:
    for poly in admin3_clipped_exploded['geometry']:
        if eachpoint.touches(poly):
            eachpointpoly = eachpoint.buffer(.0000001, 8)
            poly_coords = poly.exterior.coords
            poly_coords_length = len(poly_coords)
            polypoints = poly_coords
            for prnt in polypoints:
                if eachpointpoly.intersects(Point(prnt)):
                    point_counter[str(eachpoint)] += 1

checking  12274  admin3_outer_boundary_poly_coords


In [12]:
for pc in list(point_counter):
    if point_counter[pc] != 2:
        del point_counter[pc]

In [13]:
point_counter_df = pd.DataFrame( list(point_counter.items()), columns=['point', 'count']) #.to_csv('./max/point_counter.psv')

In [14]:
point_counter_gdf = gpd.GeoDataFrame(data=point_counter_df, geometry=[loads(p) for p in point_counter_df['point']], crs='EPSG:4326')

In [15]:
line_splitter_list = list()

for pointy in point_counter_gdf['point']:
    pointy = loads(pointy)
    closest1, closest2 = nearest_points(pointy, uganda['geometry'].iloc[0].exterior)
    # exterior!  these are all points inside a polygon, so i kept getting the input as BOTH 
    # output closest points. :/ we're trying to hit the outer edge, so we need exterior
    # took me ~2 hours to figure that out

    point_list = [closest1, closest2]
    split_line = LineString(point_list)
    split_line = scale(split_line, xfact=1.25, yfact=1.25, origin=split_line.centroid)
    if split_line.length > 0.0000001:
        line_splitter_list.append(split_line)

In [16]:
line_splitter_gdf = gpd.GeoDataFrame(pd.DataFrame(), geometry=line_splitter_list, crs={'init': 'epsg:4326'})

In [17]:
admin3_outer_boundary_gdf = gpd.GeoDataFrame(pd.DataFrame(), geometry=[admin3_outer_boundary_poly], crs={'init': 'epsg:4326'})

In [18]:
holies_gdf = explode_geometries(gpd.overlay(uganda, admin3_outer_boundary_gdf, how='symmetric_difference'))

In [19]:
split_holes_list = list()
for line in line_splitter_gdf['geometry']:
    for hole in holies_gdf['geometry']:

        if line.intersects(hole):
            split_holes_list.append(cut_polygon_by_line(hole, snap(line, hole, tolerance=.001)))

In [20]:
split_list = list()
for polys in split_holes_list:
    for p in polys:
        split_list.append(p)

In [21]:
split_list_gdf = gpd.GeoDataFrame(pd.DataFrame(), geometry=[loads(s) for s in split_list])
split_list_gdf.to_file('./split_list.shp')